<a href="https://colab.research.google.com/github/cesargar1507/DatasetsUB/blob/main/Copy_of_TFM_MOD(RevisadoF2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(dplyr)
library(readr)
install.packages("caret")
library(caret)
install.packages("ggplot2")
library(ggplot2)
install.packages("randomForest")
library(randomForest)
library(lubridate)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’




In [ ]:

# Instalar y cargar las bibliotecas necesarias
install.packages("googledrive")
library(googledrive)

In [ ]:
# Autenticarse con Google Drive
drive_auth()

Is it OK to cache OAuth access credentials in the folder ~/.cache/gargle
between R sessions?
1: Yes
2: No


In [ ]:
# Especificar el ID del archivo y la ruta de destino
file_id <- "https://drive.google.com/file/d/1c8ATZ5VxKEo1hCmjyALNGkxGY_KmI-xx/view?usp=drivesdk"  # Reemplaza este ID con el ID de tu archivo
drive_download(as_id(file_id), path = "archivo.csv", overwrite = TRUE)

In [ ]:
# Leer el archivo CSV desde la URL
urlAge <- "https://raw.githubusercontent.com/cesargar1507/DatasetsUB/main/machine_age.csv"
urlMechanic <- "https://raw.githubusercontent.com/cesargar1507/DatasetsUB/main/mechanic_antiquity.csv"
df <- read_csv("archivo.csv", show_col_types = FALSE)
df_age <- read_csv(urlAge, show_col_types = FALSE)
df_mechanic <- read_csv(urlMechanic, show_col_types = FALSE)

In [ ]:
summary(df)

In [ ]:
head(df,5)

# **LIMPIEZA DE DATOS**

In [ ]:
sapply(df, function(x) sum(is.na(x)))
#Fiabilidad suficiente para continuar con el trabajo

In [ ]:
# Remover el símbolo de dólar y las comas, y convertir a float
df$parts_cost <- as.numeric(gsub("[\\$,()]", "", df$parts_cost))

In [ ]:
# Convertir la columna 'Date' al formato adecuado
df$date <- as.Date(df$date, format="%m/%d/%Y")  # Ajusta el formato según tus datos
# Ordenar el DataFrame por fecha
df <- df %>%
  arrange(date)

In [ ]:
mean(df$downtime_gross < 0)

In [ ]:
# Filtrar registros donde downtimeGross no sea negativo
df <- df %>%
  filter(downtime_gross >= 0)

In [ ]:
mean(df$repair_hours < 0)

In [ ]:
#Eliminar columnas que no seran utilizadas
df <- select(df, -problem_code, -failure_code, -cause_code, -action_code, -pit_coverage, -real_downtime, -repair_hours)
#Se han eliminado codigos y variables calculadas

In [ ]:
# Calcular la diferencia en días entre fechas consecutivas
df <- df %>%
  group_by(equipment) %>%
  mutate(Days_Between_Failures = difftime(date, lag(date), units = "days"))

# Reemplazar los valores NA en Days_Between_Failures
df$Days_Between_Failures[is.na(df$Days_Between_Failures)] <- as.numeric(difftime(df$date[is.na(df$Days_Between_Failures)], as.Date("2023-01-01"), units = "days"))

In [ ]:
# Convertir 'Days_Between_Failures' a numeric
df$Days_Between_Failures <- as.numeric(df$Days_Between_Failures, units = "days")

In [ ]:
#Cambio de nombre de mecánico cuyo nombre está escrito de dos formas
df <- df %>%
  mutate(mechanic = ifelse(mechanic == "GALEAS PEREZ EDWIN ROGELIO", "Edwin Rogelio Galeas Perez", mechanic))

In [ ]:
# Calcular la tabla de frecuencias de mechanic
freq_mechanic <- table(df$mechanic)

# Ordenar las frecuencias de menor a mayor
freq_sorted_mechanic <- sort(freq_mechanic)

# Calcular la frecuencia relativa acumulada
freq_relative_mechanic <- cumsum(freq_sorted_mechanic) / sum(freq_sorted_mechanic)

# Definir umbral de frecuencia acumulada
umbral <- 0.2

# Identificar categorías con frecuencia relativa acumulada menor que el umbral
categorias_baja_frecuencia_mechanic <- names(freq_sorted_mechanic[freq_relative_mechanic < umbral])

# Reemplazar categorías con baja frecuencia por "Otros"
df$mechanic[df$mechanic %in% categorias_baja_frecuencia_mechanic] <- "Otros"

In [ ]:
# Calcular la tabla de frecuencias de problem_desc
freq_problem <- table(df$problem_desc)

# Ordenar las frecuencias de menor a mayor
freq_sorted_problem <- sort(freq_problem)

# Calcular la frecuencia relativa acumulada
freq_relative_problem <- cumsum(freq_sorted_problem) / sum(freq_sorted_problem)

# Definir umbral de frecuencia acumulada
umbral <- 0.2

# Identificar categorías con frecuencia relativa acumulada menor que el umbral
categorias_baja_frecuencia_problem <- names(freq_sorted_problem[freq_relative_problem < umbral])

# Reemplazar categorías con baja frecuencia por "Otros"
df$problem_desc[df$problem_desc %in% categorias_baja_frecuencia_problem] <- "Otros"

In [ ]:
# Calcular la tabla de frecuencias de failure_desc
freq_failure <- table(df$failure_desc)

# Ordenar las frecuencias de menor a mayor
freq_sorted_failure <- sort(freq_failure)

# Calcular la frecuencia relativa acumulada
freq_relative_failure <- cumsum(freq_sorted_failure) / sum(freq_sorted_failure)

# Definir umbral de frecuencia acumulada
umbral <- 0.2

# Identificar categorías con frecuencia relativa acumulada menor que el umbral
categorias_baja_frecuencia_failure <- names(freq_sorted_failure[freq_relative_failure < umbral])

# Reemplazar categorías con baja frecuencia por "Otros"
df$failure_desc[df$failure_desc %in% categorias_baja_frecuencia_failure] <- "Otros"

In [ ]:
#Convertir horas a minutos
df$downtime_gross <-df$downtime_gross*60

In [ ]:
write.csv(df,"arhivo2.csv")

# **ANALISIS DE VARIABLES**

In [ ]:
dim(df)

**Variable "Operación"**

In [ ]:
# Calcular la frecuencia de cada operación
operation_freq <- df %>%
  group_by(operation) %>%
  summarise(count = n()) %>%
  arrange(desc(count))

# Calcular la frecuencia relativa
operation_freq <- operation_freq %>%
  mutate(relative_freq = count / sum(count))

operation_freq

In [ ]:
# Ordenar los datos por frecuencia descendente
operation_freq <- operation_freq %>%
  arrange(desc(count))

options(repr.plot.width=17, repr.plot.height=10)

ggplot(operation_freq, aes(y = reorder(operation, count), x = count)) +
  geom_bar(stat = "identity", fill = 'skyblue') +
  geom_text(aes(label = paste0(round(relative_freq * 100, 1), "%")), vjust = -0.5, hjust = 0, size = 5) +
  labs(title = "Frecuencia relativa de operaciones con más fallos",
       y = "Operación",
       x = "Frecuencia") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1, size = 12), # Cambia el tamaño del texto del eje x
    axis.text.y = element_text(size = 12),                         # Cambia el tamaño del texto del eje y
    axis.title.x = element_text(size = 14),                        # Cambia el tamaño del título del eje x
    axis.title.y = element_text(size = 14),                        # Cambia el tamaño del título del eje y
    plot.title = element_text(size = 16, hjust = 0.5)              # Cambia el tamaño del título del gráfico y lo centra
  )

In [ ]:
operation_freq <- operation_freq %>%
  mutate(failure_percentage = (count / sum(count)) * 100)

In [ ]:
operation_freq <- operation_freq %>%
  mutate(failure_group = cut(failure_percentage,
                             breaks = c(-Inf, 1, 2, 3, 5, Inf),
                             labels = c("Menos del 1%", "1%-2%", "2%-3%", "3%-5%", "Más del 5%")))

In [ ]:
operation_freq

**Variable "problem_desc"**

In [ ]:
# Calcular la frecuencia de problemas de maquinaria
problem_freq <- df %>%
  group_by(problem_desc) %>%
  summarise(count = n()) %>%
  arrange(desc(count))

# Calcular la frecuencia relativa
problem_freq <- problem_freq %>%
  mutate(relative_freq = count / sum(count))

problem_freq

In [ ]:
# Ordenar los datos por frecuencia descendente
problem_freq <- problem_freq %>%
  arrange(desc(count))

options(repr.plot.width=17, repr.plot.height=6)

ggplot(problem_freq, aes(y = reorder(problem_desc, count), x = count)) +
  geom_bar(stat = "identity", fill = 'skyblue') +
  geom_text(aes(label = paste0(round(relative_freq * 100, 1), "%")), vjust = -0.5, hjust = 0) +
  labs(title = "Frecuencia relativa de problemas de maquinaria",
       y = "Problema",
       x = "Cantidad") +
  theme_minimal() +
   theme(
    axis.text.x = element_text(angle = 45, hjust = 1, size = 12), # Cambia el tamaño del texto del eje x
    axis.text.y = element_text(size = 12),                         # Cambia el tamaño del texto del eje y
    axis.title.x = element_text(size = 14),                        # Cambia el tamaño del título del eje x
    axis.title.y = element_text(size = 14),                        # Cambia el tamaño del título del eje y
    plot.title = element_text(size = 16, hjust = 0.5)              # Cambia el tamaño del título del gráfico y lo centra
  )

**Variable "failure_desc"**

In [ ]:
# Calcular la frecuencia de fallos de maquinaria
failure_freq <- df %>%
  group_by(failure_desc) %>%
  summarise(count = n()) %>%
  arrange(desc(count))

# Calcular la frecuencia relativa
failure_freq <- failure_freq %>%
  mutate(relative_freq = count / sum(count))

failure_freq

In [ ]:
# Ordenar los datos por frecuencia descendente
failure_freq <- failure_freq %>%
  arrange(desc(count))

options(repr.plot.width=20, repr.plot.height=6)

ggplot(failure_freq, aes(y = reorder(failure_desc, count), x = count)) +
  geom_bar(stat = "identity", fill = 'skyblue') +
  geom_text(aes(label = paste0(round(relative_freq * 100, 1), "%")), vjust = -0.5, hjust = 0, size = 5) +  # Etiquetas de porcentaje acumulado
  labs(title = "Frecuencia relativa de fallos",
       y = "Falla",
       x = "Cantidad") +
  theme_minimal() +
   theme(
    axis.text.x = element_text(angle = 45, hjust = 1, size = 12), # Cambia el tamaño del texto del eje x
    axis.text.y = element_text(size = 12),                         # Cambia el tamaño del texto del eje y
    axis.title.x = element_text(size = 14),                        # Cambia el tamaño del título del eje x
    axis.title.y = element_text(size = 14),                        # Cambia el tamaño del título del eje y
    plot.title = element_text(size = 16, hjust = 0.5)              # Cambia el tamaño del título del gráfico y lo centra
  )

**Variable "cause_desc"**

In [ ]:
# Calcular la frecuencia de causas de fallo
cause_freq <- df %>%
  group_by(cause_desc) %>%
  summarise(count = n()) %>%
  arrange(desc(count))

# Calcular la frecuencia relativa
cause_freq <- cause_freq %>%
  mutate(relative_freq = count / sum(count))

cause_freq

In [ ]:
# Ordenar los datos por frecuencia descendente
cause_freq <- cause_freq %>%
  arrange(desc(count))

options(repr.plot.width=20, repr.plot.height=9)

ggplot(cause_freq, aes(y = reorder(cause_desc, count), x = count)) +
  geom_bar(stat = "identity", fill = 'skyblue') +
  geom_text(aes(label = paste0(round(relative_freq * 100, 1), "%")), vjust = -0.5, hjust = 0,  size = 4) +  # Etiquetas de porcentaje acumulado
  labs(title = "Frecuencia relativa de causas de fallos",
       y = "Causa",
       x = "Cantidad") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(size = 15, hjust = 0),
    axis.title.x = element_text(size = 15),
    axis.title.y = element_text(size = 15),
    plot.title = element_text(size = 16, hjust = 0.5)
  )

In [ ]:
# Asignar grupos de frecuencia
cause_freq <- cause_freq %>%
  mutate(failure_group = case_when(
    relative_freq > 0.05 ~ "Más del 5%",
    relative_freq > 0.03 ~ "3%-5%",
    relative_freq > 0.02 ~ "2%-3%",
    relative_freq > 0.01 ~ "1%-2%",
    TRUE ~ "Menos del 1%"
  ))

# Ordenar los datos por frecuencia descendente
cause_freq <- cause_freq %>%
  arrange(desc(count))
cause_freq

**Variable "action_desc"**

In [ ]:
# Calcular la frecuencia de acciones correctivas
action_freq <- df %>%
  group_by(action_desc) %>%
  summarise(count = n()) %>%
  arrange(desc(count))

# Calcular la frecuencia relativa
action_freq <- action_freq %>%
  mutate(relative_freq = count / sum(count))

action_freq

In [ ]:
# Ordenar los datos por frecuencia descendente
action_freq <- action_freq %>%
  arrange(desc(count))

options(repr.plot.width=20, repr.plot.height=9)

ggplot(action_freq, aes(y = reorder(action_desc, count), x = count)) +
  geom_bar(stat = "identity", fill = 'skyblue') +
  geom_text(aes(label = paste0(round(relative_freq * 100, 1), "%")),
            vjust = -0.5, hjust = 0, size = 4) +  # Ajuste del tamaño del texto
  labs(title = "Frecuencia relativa de principales acciones correctivas",
       y = "Acción Correctiva",
       x = "Cantidad") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(size = 15),
    axis.title.x = element_text(size = 15),
    axis.title.y = element_text(size = 15, hjust = 1),
    plot.title = element_text(size = 16, hjust = 0.5)
  )

**Variable "parts cost"**

In [ ]:
# Crear el boxplot de costo de partes
ggplot(df, aes(y = parts_cost)) +
  geom_boxplot(fill = 'skyblue', color = 'black') +
  labs(title = 'Boxplot de parts cost',
       y = 'Costo') +
  theme_minimal()

In [ ]:
ggplot(df, aes(y = parts_cost)) +
  geom_boxplot(fill = 'skyblue', color = 'black', outlier.shape = NA) +
  labs(title = 'Boxplot de parts cost',
       y = 'Costo') +
  theme_minimal() +
  ylim(0, 50)

In [ ]:
mean(df$parts_cost, na.rm = TRUE)      # Media
median(df$parts_cost, na.rm = TRUE)    # Mediana
sd(df$parts_cost, na.rm = TRUE)        # Desviación estándar
var(df$parts_cost, na.rm = TRUE)        # Varianza
range(df$parts_cost, na.rm = TRUE)      # Rango

In [ ]:
# Histograma de 'parts cost'
options(repr.plot.width=9, repr.plot.height=9)

hist_parts <- ggplot(df, aes(x = parts_cost)) +
  geom_histogram(binwidth = 10, fill = "green", color = "black", alpha = 0.7) +
  labs(title = "Histograma de Downtime Bruto",
       x = "Costo",
       y = "Frecuencia") +
  theme_minimal()

# Mostrar histogramas
print(hist_parts)

**Variable waiting_hours**

In [ ]:
ggplot(df, aes(y = waiting_hours)) +
  geom_boxplot(fill = 'skyblue', color = 'black', outlier.shape = NA) +
  labs(title = 'Boxplot de waiting_hours',
       y = 'Horas') +
  theme_minimal() +
  ylim(0, 50)

# **ANALISIS DE VARIABLE OBJETIVO "DOWNTIME GROSS"**

In [ ]:
summary(df$downtime_gross)

In [ ]:
# Histograma de 'downtimeGross'
hist_downtimeGross <- ggplot(df, aes(x = downtime_gross)) +
  geom_histogram(binwidth = 10, fill = "green", color = "black", alpha = 0.7) +
  labs(title = "Histograma de Downtime Bruto",
       x = "Downtime Bruto (minutos)",
       y = "Frecuencia") +
  theme_minimal()

# Mostrar histogramas
print(hist_downtimeGross)

In [ ]:
# Crear el boxplot de downtime gross
options(repr.plot.width=10, repr.plot.height=7)
ggplot(df, aes(y = downtime_gross)) +
  geom_boxplot(fill = 'skyblue', color = 'black') +
  labs(title = 'Boxplot de downtime gross',
       y = 'Minutos') +
  theme_minimal() +
  theme(
    axis.text.y = element_text(size = 12), # Tamaño del texto del eje y
    axis.title.y = element_text(size = 14), # Tamaño del título del eje y
    plot.title = element_text(size = 16)    # Tamaño del título del gráfico
  )

In [ ]:
# Crear el boxplot de downtime gross
ggplot(df, aes(y = downtime_gross)) +
  geom_boxplot(fill = 'skyblue', color = 'black', outlier.shape = NA) +
  labs(title = 'Boxplot de downtime gross',
       y = 'Minutos') +
  theme_minimal() + coord_cartesian(ylim = c(0, 300))

In [ ]:
# Calcular el umbral de outliers (95% percentil)
outlier_threshold <- quantile(df$downtime_gross, 0.95)

# Identificar outliers
df$outlier <- ifelse(df$downtime_gross > outlier_threshold, TRUE, FALSE)

# Convertir la columna 'Date' a formato de fecha
df$date <- as.Date(df$date)

# Extraer solo el mes (sin el año) de la fecha
df <- df %>%
  mutate(Month = format(date, "%B")) # Nombre del mes

# Agrupar por mes y calcular la frecuencia acumulada de outliers
df_outliers <- df %>%
  group_by(Month) %>%
  summarise(Outliers = sum(outlier), .groups = 'drop') %>%
  arrange(match(Month, month.name)) # Ordenar por el orden de los meses

# Convertir la columna 'Month' a factor con el orden correcto
df_outliers$Month <- factor(df_outliers$Month, levels = month.name)

# Crear un gráfico de barras de la frecuencia acumulada de outliers por mes
ggplot(df_outliers, aes(x = Month, y = Outliers)) +
  geom_bar(stat = "identity", fill = "#1f77b4", color = 'black') + # Usar un color fijo
  labs(title = 'Frecuencia Acumulada de Outliers de Downtime Gross por Mes',
       x = 'Mes',
       y = 'Cantidad de Outliers') +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1, size = 12), # Tamaño y ángulo del texto del eje x
    axis.title.x = element_text(size = 14),                      # Tamaño del título del eje x
    axis.title.y = element_text(size = 14),                      # Tamaño del título del eje y
    plot.title = element_text(size = 16)                         # Tamaño del título del gráfico
  )

In [ ]:
# Calcular el umbral de outliers (95% percentil)
outlier_threshold <- quantile(df$downtime_gross, 0.95)

# Identificar outliers en el conjunto de datos original
df$outlier <- ifelse(df$downtime_gross > outlier_threshold, TRUE, FALSE)

# Filtrar por la operación "Hem Sleeve"
df_hem_sleeve <- df %>% filter(operation == "HEM SLEEVE")

# Contar los outliers para la operación "Hem Sleeve"
outliers_count <- sum(df_hem_sleeve$outlier)

# Imprimir el resultado
print(outliers_count)

In [ ]:
# Calcular el umbral de outliers (95% percentil)
outlier_threshold <- quantile(df$downtime_gross, 0.95)

# Identificar outliers en el conjunto de datos original
df$outlier <- ifelse(df$downtime_gross > outlier_threshold, TRUE, FALSE)

# Filtrar por la operación "Hem Sleeve"
df_hem_sleeve <- df %>% filter(operation == "HEM SLEEVE")

# Contar los outliers para la operación "Hem Sleeve"
hem_sleeve_outliers_count <- sum(df_hem_sleeve$outlier)

# Contar el total de outliers en el DataFrame original
total_outliers_count <- sum(df$outlier)

# Calcular el porcentaje de outliers de "Hem Sleeve" respecto al total de outliers
percentage_hem_sleeve_outliers <- (hem_sleeve_outliers_count / total_outliers_count) * 100

# Imprimir el resultado
print(paste("El porcentaje de outliers de 'Hem Sleeve' respecto al total es:", round(percentage_hem_sleeve_outliers, 2), "%"))

In [ ]:
# Calcular el umbral de outliers (95% percentil)
outlier_threshold <- quantile(df$downtime_gross, 0.95)

# Identificar outliers en el conjunto de datos original
df$outlier <- ifelse(df$downtime_gross > outlier_threshold, TRUE, FALSE)

# Calculamos la desviación tipica de la variable downtime_gross
sigma_dg = sd(df$downtime_gross)

# Calculamos el máximo de la variable downtime_gross
max_dg = max(df$downtime_gross)

# Agrupamos outliers por el numero de la desviación tipica
df_outliers <- df %>% filter(outlier == TRUE) %>%
                      #select(downtime_gross) %>%
                      mutate(num_sigma = floor(downtime_gross/sigma_dg)) %>%
                      group_by(num_sigma) %>%
                      summarise(num_outliers = sum(num_sigma), .groups = 'drop')

# Crear un gráfico de barras de outliers por el numero de sigmas
ggplot(df_outliers, aes(x = num_sigma, y = num_outliers)) +
  geom_bar(stat = "identity", fill = "#1f77b4", color = 'black') + # Usar un color fijo
  geom_text(aes(label = num_outliers),vjust = -0.5, hjust = +0.5, size = 4) +  # Ajuste del tamaño del texto
  labs(title = paste0('El numero de Outliers de Downtime Gross por el numero\nde la desviación tipica Sigma = ', round(sigma_dg,2) ),
       x = 'Sigmas',
       y = 'Cantidad de Outliers') +
  theme_minimal() +
  theme(
    axis.text.x = element_text(hjust = 1, size = 12), # Tamaño del texto del eje x
    axis.title.x = element_text(size = 14),           # Tamaño del título del eje x
    axis.title.y = element_text(size = 14),           # Tamaño del título del eje y
    plot.title = element_text(size = 16)              # Tamaño del título del gráfico
  )

# **PRUEBA DE HIPOTESIS**

In [ ]:
# Instalar e importar el paquete nortest
if (!require(nortest)) install.packages("nortest")
library(nortest)

In [ ]:
ad_test <- ad.test(df$downtime_gross)
print(ad_test)

In [ ]:
# Crear un gráfico Q-Q
qqnorm(df$downtime_gross)
qqline(df$downtime_gross)

In [ ]:
kruskal.test(downtime_gross ~ failure_desc, data = df)

In [ ]:
install.packages("dunn.test")
library(dunn.test)

# Realizar la prueba de Dunn
dunn.test(df$downtime_gross, df$failure_desc, kw = TRUE)

In [ ]:
# Calcular la mediana de downtime_gross por cada categoría de failure_desc
median_values <- df %>%
  group_by(failure_desc) %>%
  summarise(median_downtime = median(downtime_gross))

# Mostrar los resultados
print(median_values)

In [ ]:
# Crear un gráfico de barras de las medianas
ggplot(median_values, aes(x = reorder(failure_desc, -median_downtime), y = median_downtime)) +
  geom_bar(stat = "identity") +
  xlab("Categoría de Falla") +
  ylab("Mediana del Tiempo de Inactividad") +
  ggtitle("Mediana del Tiempo de Inactividad por Categoría de Falla") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

# **MODELOS GLM**



In [ ]:
df <- select(df, -wo_type, -outlier, -Month)
#Comparar con un glm

In [ ]:
head(df,5)

In [ ]:
# Establecer una semilla para reproducibilidad
set.seed(123)

# Definir proporciones para entrenamiento, validación y prueba
train_proportion <- 0.70
validation_proportion <- 0.20
test_proportion <- 0.10

# Calcular tamaños de cada conjunto
n <- nrow(df)
n_train <- floor(train_proportion * n)
n_validation <- floor(validation_proportion * n)
n_test <- n - n_train - n_validation  # El resto para prueba

# Crear índices para conjunto de entrenamiento, validación y prueba
trainIndex <- sample(1:n, n_train)
remaining <- setdiff(1:n, trainIndex)
validationIndex <- sample(remaining, n_validation)
testIndex <- setdiff(remaining, validationIndex)

# Crear conjuntos de entrenamiento, validación y prueba
train_data1 <- df[trainIndex, ]
validation_data1 <- df[validationIndex, ]
test_data1 <- df[testIndex, ]

# Comprobar las dimensiones de cada conjunto
cat("Dimensiones del conjunto de entrenamiento: ", dim(train_data1), "\n")
cat("Dimensiones del conjunto de validación: ", dim(validation_data1), "\n")
cat("Dimensiones del conjunto de prueba: ", dim(test_data1), "\n")

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
#Dividir en entrenamiento, test, validación por ejemplo 70,20,10
start_time <- Sys.time()
model1 <- glm(downtime_gross~. -equipment -downtime_gross -Days_Between_Failures -date -changed_pcs, data = train_data1)
end_time <- Sys.time()
training_time1 <- end_time - start_time
training_time1
# Resumen del modelo
summary(model1)

In [ ]:
# Predecir con el conjunto de validación
validation_predictions <- predict(model1, newdata = validation_data1)

# Calcular métricas de rendimiento para el conjunto de validación
validation_actuals <- validation_data1$downtime_gross
validation_rmse <- sqrt(mean((validation_predictions - validation_actuals)^2))
validation_mae <- mean(abs(validation_predictions - validation_actuals))
validation_r2 <- 1 - (sum((validation_predictions - validation_actuals)^2) / sum((validation_actuals - mean(validation_actuals))^2))

cat("Validation RMSE: ", validation_rmse, "\n")
cat("Validation MAE: ", validation_mae, "\n")
cat("Validation R-squared: ", validation_r2, "\n")

In [ ]:
# Predecir con el conjunto de prueba
test_predictions <- predict(model1, newdata = test_data1)

# Calcular métricas de rendimiento para el conjunto de prueba
test_actuals <- test_data1$downtime_gross
test_rmse <- sqrt(mean((test_predictions - test_actuals)^2))
test_mae <- mean(abs(test_predictions - test_actuals))
test_r2 <- 1 - (sum((test_predictions - test_actuals)^2) / sum((test_actuals - mean(test_actuals))^2))

cat("Test RMSE: ", test_rmse, "\n")
cat("Test MAE: ", test_mae, "\n")
cat("Test R-squared: ", test_r2, "\n")

**CALCULO DE VARIABLES**

In [ ]:
#Calcular los fallos acumulativos
df <- df %>%
  group_by(equipment) %>%
  mutate(Cumulative_Failure_Count = row_number())

In [ ]:
# Calcular el costo acumulado de mantenimiento
df <- df %>%
  group_by(equipment) %>%
  mutate(Cumulative_Maintenance_Cost = cumsum(parts_cost))

In [ ]:
# Establecer una semilla para reproducibilidad
set.seed(123)

# Definir proporciones para entrenamiento, validación y prueba
train_proportion <- 0.70
validation_proportion <- 0.20
test_proportion <- 0.10

# Calcular tamaños de cada conjunto
n <- nrow(df)
n_train <- floor(train_proportion * n)
n_validation <- floor(validation_proportion * n)
n_test <- n - n_train - n_validation  # El resto para prueba

# Crear índices para conjunto de entrenamiento, validación y prueba
trainIndex <- sample(1:n, n_train)
remaining <- setdiff(1:n, trainIndex)
validationIndex <- sample(remaining, n_validation)
testIndex <- setdiff(remaining, validationIndex)

# Crear conjuntos de entrenamiento, validación y prueba
train_data2 <- df[trainIndex, ]
validation_data2 <- df[validationIndex, ]
test_data2 <- df[testIndex, ]

# Comprobar las dimensiones de cada conjunto
cat("Dimensiones del conjunto de entrenamiento: ", dim(train_data2), "\n")
cat("Dimensiones del conjunto de validación: ", dim(validation_data2), "\n")
cat("Dimensiones del conjunto de prueba: ", dim(test_data2), "\n")

**GLM PARA PREDECIR DOWNTIME CON VARIABLES CALCULADAS**

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
start_time <- Sys.time()
model2 <- glm(downtime_gross~. -equipment -downtime_gross -date -changed_pcs, data = train_data2)
end_time <- Sys.time()
training_time2 <- end_time - start_time
training_time2
# Resumen del modelo
summary(model2)

In [ ]:
# Predecir con el conjunto de validación
validation_predictions <- predict(model2, newdata = validation_data2)

# Calcular métricas de rendimiento para el conjunto de validación
validation_actuals <- validation_data2$downtime_gross
validation_rmse <- sqrt(mean((validation_predictions - validation_actuals)^2))
validation_mae <- mean(abs(validation_predictions - validation_actuals))
validation_r2 <- 1 - (sum((validation_predictions - validation_actuals)^2) / sum((validation_actuals - mean(validation_actuals))^2))

cat("Validation RMSE: ", validation_rmse, "\n")
cat("Validation MAE: ", validation_mae, "\n")
cat("Validation R-squared: ", validation_r2, "\n")

In [ ]:
# Predecir con el conjunto de prueba (ya calculado, pero repitiendo para claridad)
test_predictions <- predict(model2, newdata = test_data2)

# Calcular métricas de rendimiento para el conjunto de prueba
test_actuals <- test_data2$downtime_gross
test_rmse <- sqrt(mean((test_predictions - test_actuals)^2))
test_mae <- mean(abs(test_predictions - test_actuals))
test_r2 <- 1 - (sum((test_predictions - test_actuals)^2) / sum((test_actuals - mean(test_actuals))^2))

cat("Test RMSE: ", test_rmse, "\n")
cat("Test MAE: ", test_mae, "\n")
cat("Test R-squared: ", test_r2, "\n")

**MODELO INCLUYENDO LA VARIABLE "MACHINE AGE"**

In [ ]:
# Renombrar la columna 'Equipment' a 'equipment' en df_age
df_age <- df_age %>% rename(equipment = Equipment)

In [ ]:
df_merged <- df %>%
  left_join(df_age %>% select(equipment, Age), by = "equipment")

In [ ]:
head(df_merged,3)

In [ ]:
# Establecer una semilla para reproducibilidad
set.seed(123)

# Definir proporciones para entrenamiento, validación y prueba
train_proportion <- 0.70
validation_proportion <- 0.20
test_proportion <- 0.10

# Calcular tamaños de cada conjunto
n <- nrow(df_merged)
n_train <- floor(train_proportion * n)
n_validation <- floor(validation_proportion * n)
n_test <- n - n_train - n_validation  # El resto para prueba

# Crear índices para conjunto de entrenamiento, validación y prueba
trainIndex <- sample(1:n, n_train)
remaining <- setdiff(1:n, trainIndex)
validationIndex <- sample(remaining, n_validation)
testIndex <- setdiff(remaining, validationIndex)

# Crear conjuntos de entrenamiento, validación y prueba
train_data3 <- df_merged[trainIndex, ]
validation_data3 <- df_merged[validationIndex, ]
test_data3 <- df_merged[testIndex, ]

# Comprobar las dimensiones de cada conjunto
cat("Dimensiones del conjunto de entrenamiento: ", dim(train_data3), "\n")
cat("Dimensiones del conjunto de validación: ", dim(validation_data3), "\n")
cat("Dimensiones del conjunto de prueba: ", dim(test_data3), "\n")

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
start_time <- Sys.time()
model3 <- glm(downtime_gross~. -equipment -downtime_gross -date -changed_pcs, data = train_data3)
end_time <- Sys.time()
training_time3 <- end_time - start_time
training_time3
# Resumen del modelo
summary(model3)

In [ ]:
# Predecir con el conjunto de validación
validation_predictions <- predict(model3, newdata = validation_data3)

# Calcular métricas de rendimiento para el conjunto de validación
validation_actuals <- validation_data3$downtime_gross
validation_rmse <- sqrt(mean((validation_predictions - validation_actuals)^2))
validation_mae <- mean(abs(validation_predictions - validation_actuals))
validation_r2 <- 1 - (sum((validation_predictions - validation_actuals)^2) / sum((validation_actuals - mean(validation_actuals))^2))

cat("Validation RMSE: ", validation_rmse, "\n")
cat("Validation MAE: ", validation_mae, "\n")
cat("Validation R-squared: ", validation_r2, "\n")

In [ ]:
# Predecir con el conjunto de prueba
test_predictions <- predict(model3, newdata = test_data3)

# Calcular métricas de rendimiento para el conjunto de prueba
test_actuals <- test_data3$downtime_gross
test_rmse <- sqrt(mean((test_predictions - test_actuals)^2))
test_mae <- mean(abs(test_predictions - test_actuals))
test_r2 <- 1 - (sum((test_predictions - test_actuals)^2) / sum((test_actuals - mean(test_actuals))^2))

cat("Test RMSE: ", test_rmse, "\n")
cat("Test MAE: ", test_mae, "\n")
cat("Test R-squared: ", test_r2, "\n")

**MODELO INCLUYENDO LA VARIABLE "MECHANIC ANTIQUITY"**

In [ ]:
# Primero, calcula la media de antiquity
mean_antiquity <- mean(df_mechanic$antiquity, na.rm = TRUE)

# Realiza el left join y luego reemplaza los NA en antiquity con la media calculada
df_merged2 <- df_merged %>%
  left_join(df_mechanic %>% select(mechanic, antiquity), by = "mechanic") %>%
  mutate(antiquity = ifelse(is.na(antiquity), mean_antiquity, antiquity))

In [ ]:
# Establecer una semilla para reproducibilidad
set.seed(123)

# Definir proporciones para entrenamiento, validación y prueba
train_proportion <- 0.70
validation_proportion <- 0.20
test_proportion <- 0.10

# Calcular tamaños de cada conjunto
n <- nrow(df_merged2)
n_train <- floor(train_proportion * n)
n_validation <- floor(validation_proportion * n)
n_test <- n - n_train - n_validation  # El resto para prueba

# Crear índices para conjunto de entrenamiento, validación y prueba
trainIndex <- sample(1:n, n_train)
remaining <- setdiff(1:n, trainIndex)
validationIndex <- sample(remaining, n_validation)
testIndex <- setdiff(remaining, validationIndex)

# Crear conjuntos de entrenamiento, validación y prueba
train_data4 <- df_merged2[trainIndex, ]
validation_data4 <- df_merged2[validationIndex, ]
test_data4 <- df_merged2[testIndex, ]

# Comprobar las dimensiones de cada conjunto
cat("Dimensiones del conjunto de entrenamiento: ", dim(train_data4), "\n")
cat("Dimensiones del conjunto de validación: ", dim(validation_data4), "\n")
cat("Dimensiones del conjunto de prueba: ", dim(test_data4), "\n")

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
start_time <- Sys.time()
model4 <- glm(downtime_gross~. -equipment -downtime_gross -date -changed_pcs, data = train_data4)
end_time <- Sys.time()
training_time4 <- end_time - start_time
training_time4
# Resumen del modelo
summary(model4)

In [ ]:
# Predecir con el conjunto de validación
validation_predictions <- predict(model4, newdata = validation_data4)

# Calcular métricas de rendimiento para el conjunto de validación
validation_actuals <- validation_data4$downtime_gross
validation_rmse <- sqrt(mean((validation_predictions - validation_actuals)^2))
validation_mae <- mean(abs(validation_predictions - validation_actuals))
validation_r2 <- 1 - (sum((validation_predictions - validation_actuals)^2) / sum((validation_actuals - mean(validation_actuals))^2))

cat("Validation RMSE: ", validation_rmse, "\n")
cat("Validation MAE: ", validation_mae, "\n")
cat("Validation R-squared: ", validation_r2, "\n")

In [ ]:
# Predecir con el conjunto de prueba
test_predictions <- predict(model4, newdata = test_data4)

# Calcular métricas de rendimiento para el conjunto de prueba
test_actuals <- test_data4$downtime_gross
test_rmse <- sqrt(mean((test_predictions - test_actuals)^2))
test_mae <- mean(abs(test_predictions - test_actuals))
test_r2 <- 1 - (sum((test_predictions - test_actuals)^2) / sum((test_actuals - mean(test_actuals))^2))

cat("Test RMSE: ", test_rmse, "\n")
cat("Test MAE: ", test_mae, "\n")
cat("Test R-squared: ", test_r2, "\n")

**MODELO INCLUYENDO LA VARIABLE "CHANGED_PCS"**

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
start_time <- Sys.time()
model5 <- glm(downtime_gross~. -equipment -downtime_gross -date -antiquity, data = train_data4)
end_time <- Sys.time()
training_time5 <- end_time - start_time
training_time5
# Resumen del modelo
summary(model5)

In [ ]:
# Predecir con el conjunto de validación
validation_predictions <- predict(model5, newdata = validation_data4)

# Calcular métricas de rendimiento para el conjunto de validación
validation_actuals <- validation_data4$downtime_gross
validation_rmse <- sqrt(mean((validation_predictions - validation_actuals)^2))
validation_mae <- mean(abs(validation_predictions - validation_actuals))
validation_r2 <- 1 - (sum((validation_predictions - validation_actuals)^2) / sum((validation_actuals - mean(validation_actuals))^2))

cat("Validation RMSE: ", validation_rmse, "\n")
cat("Validation MAE: ", validation_mae, "\n")
cat("Validation R-squared: ", validation_r2, "\n")

In [ ]:
# Predecir con el conjunto de prueba
test_predictions <- predict(model5, newdata = test_data4)

# Calcular métricas de rendimiento para el conjunto de prueba
test_actuals <- test_data4$downtime_gross
test_rmse <- sqrt(mean((test_predictions - test_actuals)^2))
test_mae <- mean(abs(test_predictions - test_actuals))
test_r2 <- 1 - (sum((test_predictions - test_actuals)^2) / sum((test_actuals - mean(test_actuals))^2))

cat("Test RMSE: ", test_rmse, "\n")
cat("Test MAE: ", test_mae, "\n")
cat("Test R-squared: ", test_r2, "\n")

**IMPLEMENTACION DE FRONTWARD**

In [ ]:
sapply(df_merged2, function(x) sum(is.na(x)))

In [ ]:
# Realizar frontward elimination
model_null <- lm(downtime_gross ~ 1, data = train_data4)
model_forward <- step(model_null,
                      scope = list(lower = model_null, upper = model5),
                      direction = "forward")
model_forward$anova

In [ ]:
formula = as.formula("downtime_gross ~ waiting_hours + changed_pcs + mechanic + action_desc +
    parts_cost + base_model + cause_desc + failure_desc + Cumulative_Maintenance_Cost +
    Cumulative_Failure_Count")

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
start_time <- Sys.time()
model6 <- glm(formula = formula, data = train_data4)
end_time <- Sys.time()
training_time6 <- end_time - start_time
training_time6
# Resumen del modelo
summary(model6)

In [ ]:
# Predecir con el conjunto de prueba
test_predictions <- predict(model6, newdata = test_data4)

# Calcular métricas de rendimiento para el conjunto de prueba
test_actuals <- test_data4$downtime_gross
test_rmse <- sqrt(mean((test_predictions - test_actuals)^2))
test_mae <- mean(abs(test_predictions - test_actuals))
test_r2 <- 1 - (sum((test_predictions - test_actuals)^2) / sum((test_actuals - mean(test_actuals))^2))

cat("Test RMSE: ", test_rmse, "\n")
cat("Test MAE: ", test_mae, "\n")
cat("Test R-squared: ", test_r2, "\n")

In [ ]:
formula2 = as.formula("downtime_gross ~ waiting_hours + changed_pcs*mechanic + action_desc +
    parts_cost + base_model + cause_desc + failure_desc + Cumulative_Maintenance_Cost +
    Cumulative_Failure_Count")

**MODELO CON INTERACCIÓN DE MECHANIC Y CHANGED PIECES**

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
# Medir el tiempo de entrenamiento
start_time <- Sys.time()
model7 <- glm(formula = formula2, data = train_data4)
end_time <- Sys.time()
training_time8 <- end_time - start_time
training_time8
# Resumen del modelo
summary(model7)

In [ ]:
# Predecir con el conjunto de prueba
test_predictions <- predict(model7, newdata = test_data4)

# Calcular métricas de rendimiento para el conjunto de prueba
test_actuals <- test_data4$downtime_gross
test_rmse <- sqrt(mean((test_predictions - test_actuals)^2))
test_mae <- mean(abs(test_predictions - test_actuals))
test_r2 <- 1 - (sum((test_predictions - test_actuals)^2) / sum((test_actuals - mean(test_actuals))^2))

cat("Test RMSE: ", test_rmse, "\n")
cat("Test MAE: ", test_mae, "\n")
cat("Test R-squared: ", test_r2, "\n")

In [ ]:
# Predecir con el conjunto de validación
validation_predictions <- predict(model7, newdata = validation_data4)

# Calcular métricas de rendimiento para el conjunto de validación
validation_actuals <- validation_data4$downtime_gross
validation_rmse <- sqrt(mean((validation_predictions - validation_actuals)^2))
validation_mae <- mean(abs(validation_predictions - validation_actuals))
validation_r2 <- 1 - (sum((validation_predictions - validation_actuals)^2) / sum((validation_actuals - mean(validation_actuals))^2))

cat("Validation RMSE: ", validation_rmse, "\n")
cat("Validation MAE: ", validation_mae, "\n")
cat("Validation R-squared: ", validation_r2, "\n")

In [ ]:
# Lista de todas las variables independientes disponibles
variables <- c("changed_pcs", "mechanic", "operation", "cause_desc",
               "problem_desc", "Age", "base_model", "action_desc",
               "failure_desc", "Days_Between_Failures",
               "Cumulative_Maintenance_Cost", "parts_cost", "waiting_hours")

# Crear una lista para almacenar los modelos y un data frame para los resultados
modelos <- list()
resultados <- data.frame(Modelo = character(), Variables = character(), Error = numeric(), R_Squared = numeric(), stringsAsFactors = FALSE)

set.seed(123)  # Fijar semilla para reproducibilidad

# Bucle para iterar y ajustar 50 modelos
for (i in 1:50) {
  # Seleccionar aleatoriamente un subconjunto de variables
  selected_vars <- sample(variables, size = sample(1:length(variables), 1))  # Tamaño del subconjunto también aleatorio

  # Definir la fórmula del modelo usando las variables seleccionadas
  formula <- as.formula(paste("downtime_gross ~", paste(selected_vars, collapse = " + ")))

  # Ajustar el modelo GLM
  modelo <- glm(formula, data = train_data4, family = gaussian())

  # Almacenar el modelo en la lista
  modelos[[paste("modelo", i, sep = "_")]] <- modelo

  # Calcular el Error Absoluto Medio (MAE)
  predicciones <- predict(modelo, validation_data4)
  mae <- mean(abs(validation_data4$downtime_gross - predicciones))

  # Calcular el R cuadrado
  ss_total <- sum((validation_data4$downtime_gross - mean(validation_data4$downtime_gross))^2)
  ss_residual <- sum((validation_data4$downtime_gross - predicciones)^2)
  r_squared <- 1 - (ss_residual / ss_total)

  # Almacenar los resultados en el data frame
  resultados <- rbind(resultados, data.frame(Modelo = paste("Modelo", i), Variables = paste(selected_vars, collapse = ", "), Error = mae, R_Squared = r_squared))
}

# Ver los resultados
resultados

In [ ]:
write.csv(resultados, "resultados.csv", row.names = TRUE)

In [ ]:
residuals <- residuals(model6)

In [ ]:
qqnorm(residuals)
qqline(residuals, col = "red")

# **MODELO RANDOM FOREST**

In [ ]:
# Entrenar el modelo de Random Forest
start_time <- Sys.time()
model_rf <- randomForest(downtime_gross ~ waiting_hours + changed_pcs + mechanic + action_desc +
    parts_cost + base_model + cause_desc + failure_desc + Cumulative_Maintenance_Cost +
    Cumulative_Failure_Count, data = train_data4, importance = TRUE)
end_time <- Sys.time()
training_time7 <- end_time - start_time
training_time7
# Ver el resumen del modelo
print(model_rf)

In [ ]:
# Predecir con el conjunto de validación
validation_predictions <- predict(model_rf, newdata = validation_data4)

# Calcular métricas de rendimiento para el conjunto de validación
validation_actuals <- validation_data4$downtime_gross
validation_rmse <- sqrt(mean((validation_predictions - validation_actuals)^2))
validation_mae <- mean(abs(validation_predictions - validation_actuals))
validation_r2 <- 1 - (sum((validation_predictions - validation_actuals)^2) / sum((validation_actuals - mean(validation_actuals))^2))

cat("Validation RMSE: ", validation_rmse, "\n")
cat("Validation MAE: ", validation_mae, "\n")
cat("Validation R-squared: ", validation_r2, "\n")

In [ ]:
# Predecir con el conjunto de prueba
test_predictions <- predict(model_rf, newdata = test_data4)

# Calcular métricas de rendimiento para el conjunto de prueba
test_actuals <- test_data4$downtime_gross
test_rmse <- sqrt(mean((test_predictions - test_actuals)^2))
test_mae <- mean(abs(test_predictions - test_actuals))
test_r2 <- 1 - (sum((test_predictions - test_actuals)^2) / sum((test_actuals - mean(test_actuals))^2))

cat("Test RMSE: ", test_rmse, "\n")
cat("Test MAE: ", test_mae, "\n")
cat("Test R-squared: ", test_r2, "\n")

In [ ]:
residuals <- test_data4$downtime_gross - test_predictions

In [ ]:
qqnorm(residuals)
qqline(residuals, col = "red")

In [ ]:
# Ver la importancia de las variables
importance(model_rf)
varImpPlot(model_rf)

In [ ]:
install.packages("nortest")
library(nortest)

In [ ]:
ad_test <- ad.test(residuals)
print(ad_test)

# MODELO SVM

In [ ]:
# Instalar el paquete e1071 si no lo tienes
install.packages("e1071")

# Cargar el paquete
library(e1071)

In [ ]:
# Medir el tiempo de entrenamiento
start_time <- Sys.time()

# Ajustar el modelo SVM
model_svm <- svm(downtime_gross ~ waiting_hours + changed_pcs + mechanic + action_desc +
    parts_cost + base_model + cause_desc + failure_desc + Cumulative_Maintenance_Cost +
    Cumulative_Failure_Count, data = train_data4, kernel = "radial", cost = 1, gamma = 0.1)

end_time <- Sys.time()
training_time_svm <- end_time - start_time

# Mostrar el tiempo de entrenamiento
print(training_time_svm)

# Ver el resumen del modelo SVM
print(model_svm)

In [ ]:
# Predecir con el conjunto de validación usando el modelo SVM
validation_predictions_svm <- predict(model_svm, newdata = validation_data4)

# Calcular métricas de rendimiento para el conjunto de validación
validation_actuals <- validation_data4$downtime_gross
validation_rmse_svm <- sqrt(mean((validation_predictions_svm - validation_actuals)^2))
validation_mae_svm <- mean(abs(validation_predictions_svm - validation_actuals))
validation_r2_svm <- 1 - (sum((validation_predictions_svm - validation_actuals)^2) / sum((validation_actuals - mean(validation_actuals))^2))

cat("Validation RMSE (SVM): ", validation_rmse_svm, "\n")
cat("Validation MAE (SVM): ", validation_mae_svm, "\n")
cat("Validation R-squared (SVM): ", validation_r2_svm, "\n")

In [ ]:
# Predecir con el conjunto de prueba usando el modelo SVM
test_predictions_svm <- predict(model_svm, newdata = test_data4)

# Calcular métricas de rendimiento para el conjunto de prueba
test_actuals <- test_data4$downtime_gross
test_rmse_svm <- sqrt(mean((test_predictions_svm - test_actuals)^2))
test_mae_svm <- mean(abs(test_predictions_svm - test_actuals))
test_r2_svm <- 1 - (sum((test_predictions_svm - test_actuals)^2) / sum((test_actuals - mean(test_actuals))^2))

cat("Test RMSE (SVM): ", test_rmse_svm, "\n")
cat("Test MAE (SVM): ", test_mae_svm, "\n")
cat("Test R-squared (SVM): ", test_r2_svm, "\n")

# **MODELO GRADIENT BOOSTING**

**Cargamos la librería gbm**

In [ ]:
install.packages('gbm')
library(gbm)

In [ ]:
dim(train_data4)
dim(validation_data4)
dim(test_data4)

In [ ]:
glimpse(test_data4)

**Numerización**

Convertimos a numérico las variables **operation, base_model, problem_desc, failure_desc, cause_desc, action_desc, mechanic** que contienen datos textuales.

In [ ]:
install.packages("superml")
library(superml)
library(scales)

In [ ]:
encoder1 <- LabelEncoder$new()
encoder2 <- LabelEncoder$new()
encoder3 <- LabelEncoder$new()
encoder4 <- LabelEncoder$new()
encoder5 <- LabelEncoder$new()
encoder6 <- LabelEncoder$new()
encoder7 <- LabelEncoder$new()

data_for_gb <- rbind(rbind(train_data4, validation_data4), test_data4)

data_for_gb$operation <- encoder1$fit_transform(data_for_gb$operation)
data_for_gb$base_model <- encoder2$fit_transform(data_for_gb$base_model)
data_for_gb$problem_desc <- encoder3$fit_transform(data_for_gb$problem_desc)
data_for_gb$failure_desc <- encoder4$fit_transform(data_for_gb$failure_desc)
data_for_gb$cause_desc <- encoder5$fit_transform(data_for_gb$cause_desc)
data_for_gb$action_desc <- encoder6$fit_transform(data_for_gb$action_desc)
data_for_gb$mechanic <- encoder7$fit_transform(data_for_gb$mechanic)

data_for_gb$changed_pcs <- rescale(data_for_gb$changed_pcs)
data_for_gb$mechanic <- rescale(data_for_gb$mechanic)
data_for_gb$operation <- rescale(data_for_gb$operation)
data_for_gb$cause_desc <- rescale(data_for_gb$cause_desc)
data_for_gb$problem_desc <- rescale(data_for_gb$problem_desc)
data_for_gb$Age <- rescale(data_for_gb$Age)
data_for_gb$base_model <- rescale(data_for_gb$base_model)
data_for_gb$action_desc <- rescale(data_for_gb$action_desc)
data_for_gb$failure_desc <- rescale(data_for_gb$failure_desc)
data_for_gb$Days_Between_Failures <- rescale(data_for_gb$Days_Between_Failures)
data_for_gb$Cumulative_Maintenance_Cost <- rescale(data_for_gb$Cumulative_Maintenance_Cost)
data_for_gb$parts_cost <- rescale(data_for_gb$parts_cost)

head(data_for_gb, 6)

In [ ]:
# Crear conjuntos de entrenamiento, validación y prueba para el modelo de Gradient Boosting

train_data5 <- data_for_gb[1:nrow(train_data4),]
validation_data5 <- data_for_gb[(nrow(train_data4)+1):(nrow(train_data4) + nrow(validation_data4)),]
test_data5 <- data_for_gb[(nrow(train_data4) + nrow(validation_data4)+1):(nrow(train_data4) + nrow(validation_data4)+nrow(test_data4)),]

dim(train_data5)
dim(validation_data5)
dim(test_data5)

In [ ]:
formula

**Modelización y Evaluación**

Entrenamos el modelo para varios valores de la máxima profundidad de árboles (max_depth) de 1 a 30.

In [ ]:
n_estimators <- 100
learning_rate <-  0.01

max_depth_list <- c()
train_rmse_list <- c()
train_mae_list <- c()
train_r2_list <- c()
val_rmse_list <-c()
val_mae_list <- c()
val_r2_list <- c()

for (max_depth in 1:30) {
 cat("\nLa profundidad maxima de árboles: ", max_depth, "\n")
 start_time <- Sys.time()
 model_gbm = gbm(downtime_gross ~ waiting_hours + changed_pcs + mechanic + action_desc +
    parts_cost + base_model + cause_desc + failure_desc + Cumulative_Maintenance_Cost +
    Cumulative_Failure_Count, data = train_data5,
                 distribution = "gaussian",
                # cv.folds = 10,
                 shrinkage = learning_rate,
                 n.trees = n_estimators,
                 interaction.depth = max_depth)
 end_time <- Sys.time()
 training_time8 <- end_time - start_time
 training_time8

 # Predecir con el conjunto de entrenamiento
 train_predictions <- predict(model_gbm, newdata = train_data5)

 # Calcular métricas de rendimiento para el conjunto de entrenamiento
 train_actuals <- train_data5$downtime_gross
 train_rmse <- sqrt(mean((train_predictions - train_actuals)^2))
 train_mae <- mean(abs(train_predictions - train_actuals))
 train_r2 <- 1 - (sum((train_predictions - train_actuals)^2) / sum((train_actuals - mean(train_actuals))^2))

 cat("Train RMSE: ", train_rmse, "\n")
 cat("Train MAE: ", train_mae, "\n")
 cat("Train R-squared: ", train_r2, "\n\n")

 # Predecir con el conjunto de validación
 validation_predictions <- predict(model_gbm, newdata = validation_data5)

 # Calcular métricas de rendimiento para el conjunto de validación
 validation_actuals <- validation_data5$downtime_gross
 validation_rmse <- sqrt(mean((validation_predictions - validation_actuals)^2))
 validation_mae <- mean(abs(validation_predictions - validation_actuals))
 validation_r2 <- 1 - (sum((validation_predictions - validation_actuals)^2) / sum((validation_actuals - mean(validation_actuals))^2))

 cat("Validation RMSE: ", validation_rmse, "\n")
 cat("Validation MAE: ", validation_mae, "\n")
 cat("Validation R-squared: ", validation_r2, "\n")

 max_depth_list[max_depth] <- max_depth
 train_rmse_list[max_depth] <- train_rmse
 train_mae_list[max_depth] <- train_mae
 train_r2_list[max_depth] <- train_r2
 val_rmse_list[max_depth] <-validation_rmse
 val_mae_list[max_depth] <- validation_mae
 val_r2_list[max_depth] <- validation_r2

}


In [ ]:
df_metricas <- data.frame(max_depth_list, train_rmse_list, train_mae_list, train_r2_list, val_rmse_list,val_mae_list, val_r2_list)
colnames(df_metricas) <- c("max_depth", "train_rmse", "train_mae", "train_r2", "val_rmse", "val_mae", "val_r2")
df_metricas

In [ ]:
ggplot(df_metricas, aes(x = max_depth)) +
    geom_line(aes(y = train_rmse, color = "Train RMSE")) +
    geom_line(aes(y = val_rmse, color = "Validation RMSE")) +
    labs(title = "RMSE por la profundidad de los Arboles para Gradient Boosting",
         x = "Profundidad máxima de árboles",
         y = "RMSE") +
    scale_color_manual(values = c("Train RMSE" = "blue", "Validation RMSE" = "red")) +
    theme_minimal()

In [ ]:
ggplot(df_metricas, aes(x = max_depth)) +
    geom_line(aes(y = train_r2, color = "Train R2")) +
    geom_line(aes(y = val_r2, color = "Validation R2")) +
    labs(title = "R2 por la profundidad de los Arboles para Gradient Boosting",
         x = "Profundidad máxima de árboles",
         y = "R2") +
    scale_color_manual(values = c("Train R2" = "blue", "Validation R2" = "red")) +
    theme_minimal()

Entrenamos el modelo con valor optimo de max_depth = 7 para diferentes valores dl numero de árboles (n_estimators)

In [ ]:
learning_rate <-  0.01
max_depth <- 7

n_estimators_list <- c()
train_rmse_list <- c()
train_mae_list <- c()
train_r2_list <- c()
val_rmse_list <-c()
val_mae_list <- c()
val_r2_list <- c()
i <- 1

for (n_estimators in seq(100, 2000, by=100)) {
 cat("\nEl numero árboles: ", n_estimators, "\n")
 start_time <- Sys.time()
 model_gbm = gbm(downtime_gross ~ waiting_hours + changed_pcs + mechanic + action_desc +
    parts_cost + base_model + cause_desc + failure_desc + Cumulative_Maintenance_Cost +
    Cumulative_Failure_Count, data = train_data5,
                 distribution = "gaussian",
                # cv.folds = 10,
                 shrinkage = learning_rate,
                 n.trees = n_estimators,
                 interaction.depth = max_depth)
 end_time <- Sys.time()
 training_time8 <- end_time - start_time
 cat("Training time: ", training_time8, "\n")

 # Predecir con el conjunto de entrenamiento
 train_predictions <- predict(model_gbm, newdata = train_data5)

 # Calcular métricas de rendimiento para el conjunto de entrenamiento
 train_actuals <- train_data5$downtime_gross
 train_rmse <- sqrt(mean((train_predictions - train_actuals)^2))
 train_mae <- mean(abs(train_predictions - train_actuals))
 train_r2 <- 1 - (sum((train_predictions - train_actuals)^2) / sum((train_actuals - mean(train_actuals))^2))

 cat("Train RMSE: ", train_rmse, "\n")
 cat("Train MAE: ", train_mae, "\n")
 cat("Train R-squared: ", train_r2, "\n\n")

 # Predecir con el conjunto de validación
 validation_predictions <- predict(model_gbm, newdata = validation_data5)

 # Calcular métricas de rendimiento para el conjunto de validación
 validation_actuals <- validation_data5$downtime_gross
 validation_rmse <- sqrt(mean((validation_predictions - validation_actuals)^2))
 validation_mae <- mean(abs(validation_predictions - validation_actuals))
 validation_r2 <- 1 - (sum((validation_predictions - validation_actuals)^2) / sum((validation_actuals - mean(validation_actuals))^2))

 cat("Validation RMSE: ", validation_rmse, "\n")
 cat("Validation MAE: ", validation_mae, "\n")
 cat("Validation R-squared: ", validation_r2, "\n")

 n_estimators_list[i] <- n_estimators
 train_rmse_list[i] <- train_rmse
 train_mae_list[i] <- train_mae
 train_r2_list[i] <- train_r2
 val_rmse_list[i] <-validation_rmse
 val_mae_list[i] <- validation_mae
 val_r2_list[i] <- validation_r2
 i <- i + 1

}

df_metricas <- data.frame(n_estimators_list, train_rmse_list, train_mae_list, train_r2_list, val_rmse_list,val_mae_list, val_r2_list)
colnames(df_metricas) <- c("n_estimators", "train_rmse", "train_mae", "train_r2", "val_rmse", "val_mae", "val_r2")
df_metricas



In [ ]:
ggplot(df_metricas, aes(x = n_estimators)) +
    geom_line(aes(y = train_rmse, color = "Train RMSE")) +
    geom_line(aes(y = val_rmse, color = "Validation RMSE")) +
    labs(title = "RMSE por el numero de los Arboles para Gradient Boosting",
         x = "Numero de árboles",
         y = "RMSE") +
    scale_color_manual(values = c("Train RMSE" = "blue", "Validation RMSE" = "red")) +
    theme_minimal()

In [ ]:
ggplot(df_metricas, aes(x = n_estimators)) +
    geom_line(aes(y = train_r2, color = "Train R2")) +
    geom_line(aes(y = val_r2, color = "Validation R2")) +
    labs(title = "R2 por el numero de los Arboles para Gradient Boosting",
         x = "Numero de árboles",
         y = "R2") +
    scale_color_manual(values = c("Train R2" = "blue", "Validation R2" = "red")) +
    theme_minimal()

In [ ]:
df_metricas[df_metricas$n_estimators==500,]

Entrenamos el modelo optimal con max_depth = 7 y n_estimators = 500

In [ ]:
formula

In [ ]:
learning_rate <-  0.01
max_depth <- 7
n_estimators <- 500

start_time <- Sys.time()
model_gbm = gbm(formula=formula, data = train_data5,
                 distribution = "gaussian",
                 #cv.folds = 10,
                 shrinkage = learning_rate,
                 n.trees = n_estimators,
                 interaction.depth = max_depth)
end_time <- Sys.time()
training_time8 <- end_time - start_time
cat("Training time: ", training_time8, "\n")

# Predecir con el conjunto de entrenamiento
train_predictions <- predict(model_gbm, newdata = train_data5)

# Calcular métricas de rendimiento para el conjunto de entrenamiento
train_actuals <- train_data5$downtime_gross
train_rmse <- sqrt(mean((train_predictions - train_actuals)^2))
train_mae <- mean(abs(train_predictions - train_actuals))
train_r2 <- 1 - (sum((train_predictions - train_actuals)^2) / sum((train_actuals - mean(train_actuals))^2))

cat("Train RMSE: ", train_rmse, "\n")
cat("Train MAE: ", train_mae, "\n")
cat("Train R-squared: ", train_r2, "\n\n")

# Predecir con el conjunto de validación
test_predictions <- predict(model_gbm, newdata = test_data5)

# Calcular métricas de rendimiento para el conjunto de validación
test_actuals <- test_data5$downtime_gross
test_rmse <- sqrt(mean((test_predictions - test_actuals)^2))
test_mae <- mean(abs(test_predictions - test_actuals))
test_r2 <- 1 - (sum((test_predictions - test_actuals)^2) / sum((test_actuals - mean(test_actuals))^2))

cat("Test RMSE: ", test_rmse, "\n")
cat("Test MAE: ", test_mae, "\n")
cat("Test R-squared: ", test_r2, "\n")

# **MODELO RED NEURAL**

In [ ]:
install.packages("tensorflow")
library(tensorflow)
install.packages("keras3")
library(keras3)
install.packages("tidyverse")
library(tidyverse)
install.packages("tidymodels")
library(tidymodels)

In [ ]:
dim(train_data4)
dim(validation_data4)
dim(test_data4)

In [ ]:
formula

**Creamos el conjunto de datos del entrenamiento y de la evaluación**

In [ ]:
encoder1 <- LabelEncoder$new()
encoder2 <- LabelEncoder$new()
encoder3 <- LabelEncoder$new()
encoder4 <- LabelEncoder$new()
encoder5 <- LabelEncoder$new()
encoder6 <- LabelEncoder$new()
encoder7 <- LabelEncoder$new()

data_for_rn <- rbind(rbind(train_data4, validation_data4), test_data4)

data_for_rn$operation <- encoder1$fit_transform(data_for_rn$operation)
data_for_rn$base_model <- encoder2$fit_transform(data_for_rn$base_model)
data_for_rn$problem_desc <- encoder3$fit_transform(data_for_rn$problem_desc)
data_for_rn$failure_desc <- encoder4$fit_transform(data_for_rn$failure_desc)
data_for_rn$cause_desc <- encoder5$fit_transform(data_for_rn$cause_desc)
data_for_rn$action_desc <- encoder6$fit_transform(data_for_rn$action_desc)
data_for_rn$mechanic <- encoder7$fit_transform(data_for_rn$mechanic)

data_for_rn <- data_for_rn[, c("waiting_hours", "changed_pcs", "mechanic", "action_desc",
                               "parts_cost", "base_model", "cause_desc", "failure_desc",
                               "Cumulative_Maintenance_Cost", "Cumulative_Failure_Count",
                               "downtime_gross")]
head(data_for_rn, 6)


In [ ]:
summary(data_for_rn)

In [ ]:
# Cambiamos los valores NA de la variable Age por la mediana
#mediana_Age <- median(data_for_rn$Age, na.rm = TRUE)
#data_for_rn$Age[is.na(data_for_rn$Age)] <- mediana_Age
#summary(data_for_rn$Age)

In [ ]:
# Crear conjuntos de entrenamiento y de prueba para el modelo Red Neural

train_data6 <- data_for_rn[1:(nrow(train_data4) + nrow(validation_data4)),]
test_data6 <- data_for_rn[(nrow(train_data4) + nrow(validation_data4)+1):(nrow(train_data4) + nrow(validation_data4)+nrow(test_data4)),]

dim(train_data6)
dim(test_data6)

In [ ]:
# Dividimos datasets por características y etiquetas para entrenar luego el modelo

X_train <- as.matrix(select(train_data6, -downtime_gross))
Y_train <- as.matrix(select(train_data6, downtime_gross))

X_test <- as.matrix(select(test_data6, -downtime_gross))
Y_test <- as.matrix(select(test_data6, downtime_gross))

print("Dimenciones del conjunto de datos de entrenamiento: ")
dim(X_train)
dim(Y_train)

print("Dimenciones del conjunto de datos de prueba: ")
dim(X_test)
dim(Y_test)

**Creamos el modelo optimo**

In [ ]:
normalizer <- layer_normalization(axis = -1L)
normalizer %>% adapt(X_train)

In [ ]:
print(normalizer$mean)

Dos capas leneales, epochs = 30

In [ ]:
model_rn_optimo <- keras_model_sequential(shape=c(10)) %>%
    normalizer() %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(1)

model_rn_optimo %>% compile(
    loss = 'mean_squared_error',
    optimizer = optimizer_adam(0.01),
    metrics = "r2_score")

summary(model_rn_optimo)

In [ ]:
history <- model_rn_optimo %>% fit(
  x = X_train,
  y = Y_train,
  validation_split = 0.2,
  verbose = 2,
  epochs = 30)

In [ ]:
plot(history)

In [ ]:
history

In [ ]:
# Predecir con el conjunto de entrenamiento
train_predictions <- predict(model_rn_optimo, X_train)

# Calcular métricas de rendimiento para el conjunto de entrenamiento
train_actuals <- Y_train #train_data6$downtime_gross
train_rmse <- sqrt(mean((train_predictions - train_actuals)^2))
train_mae <- mean(abs(train_predictions - train_actuals))
train_r2 <- 1 - (sum((train_predictions - train_actuals)^2) / sum((train_actuals - mean(train_actuals))^2))

cat("Train RMSE: ", train_rmse, "\n")
cat("Train MAE: ", train_mae, "\n")
cat("Train R-squared: ", train_r2, "\n\n")

# Predecir con el conjunto de validación
test_predictions <- predict(model_rn_optimo, X_test)

# Calcular métricas de rendimiento para el conjunto de validación
test_actuals <- Y_test #test_data6$downtime_gross
test_rmse <- sqrt(mean((test_predictions - test_actuals)^2))
test_mae <- mean(abs(test_predictions - test_actuals))
test_r2 <- 1 - (sum((test_predictions - test_actuals)^2) / sum((test_actuals - mean(test_actuals))^2))

cat("Test RMSE: ", test_rmse, "\n")
cat("Test MAE: ", test_mae, "\n")
cat("Test R-squared: ", test_r2, "\n")

**Pruebas de modelos con parametros diferentes**

Dos capas lineales, epochs = 100, loss = mean_squared_error, learning_rate = 0.01

In [ ]:
model_rn <- keras_model_sequential(shape=c(10)) %>%
    normalizer() %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(1)

model_rn %>% compile(
    loss = 'mean_squared_error',
    optimizer = optimizer_adam(0.01),
    metrics = "r2_score")

summary(model_rn)

In [ ]:
history <- model_rn %>% fit(
  x = X_train,
  y = Y_train,
  validation_split = 0.2,
  verbose = 2,
  epochs = 100)

In [ ]:
plot(history)

In [ ]:
history

Dos capas lineale, epochs = 30, loss = mean_absolute_error, learning_rate = 0.01

In [ ]:
model_rn1 <- keras_model_sequential(shape=c(10)) %>%
    normalizer() %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(1)

model_rn1 %>% compile(
    loss = 'mean_absolute_error',
    optimizer = optimizer_adam(0.01),
    metrics = "r2_score")

summary(model_rn1)

In [ ]:
history1 <- model_rn1 %>% fit(
  x = X_train,
  y = Y_train,
  validation_split = 0.2,
  verbose = 2,
  epochs = 30)

In [ ]:
history1

In [ ]:
plot(history1)

Una capa lineal, leraning_rate = 0.01

In [ ]:
model_rn2 <- keras_model_sequential(shape=c(10)) %>%
    normalizer() %>%
    layer_dense(1)

model_rn2 %>% compile(
    loss = 'mean_squared_error',
    optimizer = optimizer_adam(0.01),
    metrics = "r2_score")

summary(model_rn2)

In [ ]:
history2 <- model_rn2 %>% fit(
  x = X_train,
  y = Y_train,
  validation_split = 0.2,
  verbose = 2,
  epochs = 30)

In [ ]:
plot(history2)
history2

Dos capas lineale, epochs = 30, loss = mean_squared_error, learning_rate = 0.001

In [ ]:
model_rn3 <- keras_model_sequential(shape=c(10)) %>%
    normalizer() %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(1)

model_rn3 %>% compile(
    loss = 'mean_squared_error',
    optimizer = optimizer_adam(0.001),
    metrics = "r2_score")

summary(model_rn3)

In [ ]:
history3 <- model_rn3 %>% fit(
  x = X_train,
  y = Y_train,
  validation_split = 0.2,
  verbose = 2,
  epochs = 50)

plot(history3)
history3

Una capa lineal, loss = mean_absolute_error, learning_rate = 0.001

In [ ]:
model_rn4 <- keras_model_sequential(shape=c(10)) %>%
    normalizer() %>%
    layer_dense(1)

model_rn4 %>% compile(
    loss = 'mean_absolute_error',
    optimizer = optimizer_adam(0.001),
    metrics = "r2_score")

summary(model_rn4)

In [ ]:
history4 <- model_rn4 %>% fit(
  x = X_train,
  y = Y_train,
  validation_split = 0.2,
  verbose = 2,
  epochs = 50)

plot(history4)
history4

Dos capas lineales, epochs = 30, loss = mean_squared_error, learning_rate = 0.001

In [ ]:
model_rn5 <- keras_model_sequential(shape=c(10)) %>%
    normalizer() %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(1)

model_rn5 %>% compile(
    loss = 'mean_squared_error',
    optimizer = optimizer_adam(0.001),
    metrics = "r2_score")

summary(model_rn5)

In [ ]:
history5 <- model_rn5 %>% fit(
  x = X_train,
  y = Y_train,
  validation_split = 0.2,
  verbose = 2,
  epochs = 30)

  plot(history5)
  history5

Tres capas lineales, loss = mean_squared_error, learning_rate = 0.01, epoch = 30

In [ ]:
model_rn6 <- keras_model_sequential(shape=c(10)) %>%
    normalizer() %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(1)

model_rn6 %>% compile(
    loss = 'mean_squared_error',
    optimizer = optimizer_adam(0.01),
    metrics = "r2_score")

summary(model_rn6)

In [ ]:
history6 <- model_rn6 %>% fit(
  x = X_train,
  y = Y_train,
  validation_split = 0.2,
  verbose = 2,
  epochs = 30)

  plot(history6)
  history6

Tres capas lineales, loss = mean_squared_error, learning_rate = 0.001, epoch = 30

In [ ]:
model_rn7 <- keras_model_sequential(shape=c(10)) %>%
    normalizer() %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(64, activation = 'relu') %>%
    layer_dense(1)

model_rn7 %>% compile(
    loss = 'mean_squared_error',
    optimizer = optimizer_adam(0.001),
    c("r2_score", "mean_absolute_error")

summary(model_rn7)

In [ ]:
history7 <- model_rn7 %>% fit(
  x = X_train,
  y = Y_train,
  validation_split = 0.3,
  verbose = 2,
  epochs = 30)

  plot(history7)
  history7